In [2]:
#1. 라이브러리 
# 사용할 라이브러리 불러오기
import datetime
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt   

from tensorflow.keras import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, AveragePooling2D, Dropout, Input

In [3]:
# 분류 : 꽃 종류(5종) 분류
num_class = 5

In [5]:
# AlexNet 모델 정의
class AlexNet(Sequential):
    def __init__(self, input_shape, num_class):
        super().__init__()

        self.add(Input(input_shape))
        self.add(Conv2D(96, kernel_size=(11,11), strides=4, padding='valid', activation='relu',
                        kernel_initializer='he_normal'))
        self.add(MaxPooling2D(pool_size=(3,3), strides=(2,2), padding='valid', data_format=None))
        self.add(Conv2D(256, kernel_size=(5,5), strides=1, padding='same', activation='relu',
                        kernel_initializer='he_normal'))
        self.add(MaxPooling2D(pool_size=(3,3), strides=(2,2), padding='valid', data_format=None))
        
        self.add(Conv2D(384, kernel_size=(3,3), strides=1, padding='same', activation='relu',
                        kernel_initializer='he_normal'))

        self.add(Conv2D(384, kernel_size=(3,3), strides=1, padding='same', activation='relu',
                        kernel_initializer='he_normal'))
        self.add(Conv2D(256, kernel_size=(3,3), strides=1, padding='same', activation='relu',
                        kernel_initializer='he_normal'))

        self.add(MaxPooling2D(pool_size=(3,3), strides=(2,2), padding='valid', data_format=None))
        
        self.add(Flatten())
        self.add(Dense(4096, activation='relu'))
        self.add(Dense(4096, activation='relu'))
        self.add(Dense(1000, activation='relu'))
        self.add(Dense(num_class, activation='softmax'))
        
        self.compile(optimizer=tf.keras.optimizers.Adam(0.001),
                     loss=categorical_crossentropy,
                     metrics=['accuracy'])

In [6]:
# 모델 생성 
# AlexNet 모델 생성 및 요약 보기
AlexNetModel = AlexNet((100,100,3), num_class)
AlexNetModel.summary()

Model: "alex_net"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 23, 23, 96)     │        34,944 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 11, 11, 96)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 11, 11, 256)    │       614,656 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 5, 5, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 5, 5, 384)      │       885,120 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 5, 5, 384)      │     1,327,488 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 5, 5, 256)      │       884,992 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 2, 2, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 1024)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 4096)           │     4,198,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 4096)           │    16,781,312 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1000)           │     4,097,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 5)              │         5,005 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 28,828,917 (109.97 MB)

 Trainable params: 28,828,917 (109.97 MB)

 Non-trainable params: 0 (0.00 B)

In [7]:
# 데이터 로드 및 전처리 작업.... 
flowers = tf.keras.utils.get_file(
    'flower_photos',
    'https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz',
    untar=True
)

In [8]:
# flowers+"\\flower_photos"
%pip install split-folders

In [9]:
# splitfolders를 이용하여 훈련데이터와 검증데이터로 나눔.
import splitfolders
splitfolders.ratio(flowers+"\\flower_photos", output="flower_photos", seed=100, ratio=(.8,.2))

Copying files: 3670 files [00:03, 1222.74 files/s]


In [10]:
BATCH_SIZE=256
EPOCHS=100
train_dir = "./flower_photos/train/"
valid_dir = "./flower_photos/val/"

In [11]:
# 이미지 전처리
train = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1.0/255,
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1
)
train_generator = train.flow_from_directory(
    train_dir,
    target_size=(100,100),
    color_mode='rgb',
    batch_size=BATCH_SIZE,
    seed=1,
    shuffle=True,
    class_mode="categorical"
)
valid = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1.0/255)
valid_generator = valid.flow_from_directory(
    valid_dir,
    target_size=(100,100),
    color_mode='rgb',
    batch_size=BATCH_SIZE,
    seed=7,
    shuffle=True,
    class_mode="categorical"
)
train_num = train_generator.samples
valid_num = valid_generator.samples

Found 2934 images belonging to 5 classes.
Found 736 images belonging to 5 classes.


In [12]:
%load_ext tensorboard
# # AlexNetModel 적용!
# 1. 동작 log를 저장할 디렉터리 지정
log_dir = "./img/log2_AlexNet2/"   # 훈련 데이터를 ./img/log1/ 디렉터리 저장. 
# 2. tensorboard 객체 생성
tensorboard_callback = tf.keras.callbacks.TensorBoard(
    log_dir=log_dir,    # 로그가 저장된 디렉터리
    histogram_freq=1,   # 에포트마다 출력을 히스토그램으로 기록
    profile_batch=0,    # 비활성화. 시스템자원 및 시간 사용에 대한 관리 
)
# 3. 모델 학습(텐서보드 callback을 지정)
AlexNetModel.fit(train_generator, epochs=100,
          steps_per_epoch=train_num // BATCH_SIZE,  # 정수 나누기
          validation_data=valid_generator,          # 검증 이미지 데이터
          validation_steps=valid_num // BATCH_SIZE, # 
          callbacks=[tensorboard_callback],         # callbacks 으로 텐서보드 지정(*)
          verbose=1 
)

d:\kim\TensorFlow\venv\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 35s 3s/step - accuracy: 0.2082 - loss: 19.8370 - val_accuracy: 0.2637 - val_loss: 1.5974
Epoch 2/100
 1/11 ━━━━━━━━━━━━━━━━━━━━ 31s 3s/step - accuracy: 0.2695 - loss: 1.6037

d:\kim\TensorFlow\venv\Lib\site-packages\keras\src\trainers\epoch_iterator.py:107: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


11/11 ━━━━━━━━━━━━━━━━━━━━ 7s 435ms/step - accuracy: 0.2695 - loss: 1.6037 - val_accuracy: 0.2383 - val_loss: 1.6027
Epoch 3/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 18s 2s/step - accuracy: 0.2468 - loss: 1.5980 - val_accuracy: 0.2109 - val_loss: 1.8232
Epoch 4/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 3s 225ms/step - accuracy: 0.2617 - loss: 1.7365 - val_accuracy: 0.2188 - val_loss: 1.6661
Epoch 5/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 16s 1s/step - accuracy: 0.2903 - loss: 1.5687 - val_accuracy: 0.3086 - val_loss: 1.4750
Epoch 6/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 3s 241ms/step - accuracy: 0.2852 - loss: 1.4707 - val_accuracy: 0.3105 - val_loss: 1.4261
Epoch 7/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 17s 1s/step - accuracy: 0.3588 - loss: 1.4040 - val_accuracy: 0.4121 - val_loss: 1.3196
Epoch 8/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 3s 222ms/step - accuracy: 0.3559 - loss: 1.3464 - val_accuracy: 0.3945 - val_loss: 1.3049
Epoch 9/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 16s 1s/step - accuracy: 0.4347 - loss: 1.3034 - val_accuracy: 0.5000 - val_l

In [ ]:
class_names = ['daisy','dandelion','roses','sunflowers','tulips']
validation, label_batch = next(iter(valid_generator))
prediction_values = AlexNetModel.predict(validation)
prediction_values = np.argmax(prediction_values, axis=1)

fig = plt.figure(figsize=(12,8))
fig.subplots_adjust(left=0, right=1, bottom=0, top=1, hspace=0.05, wspace=0.05)

for i in range(8):
    ax = fig.add_subplot(2, 4, i+1, xticks=[], yticks=[])
    ax.imshow(validation[i,:], cmap=plt.cm.gray_r, interpolation='nearest')
    
    if prediction_values[i] == np.argmax(label_batch[i]):
        ax.text(3, 17, class_names[prediction_values[i]], color='yellow', fontsize=14)
    else:
        ax.text(3, 17, class_names[prediction_values[i]], color='red', fontsize=14)